In [1]:
# Create monthly report url

# To run this directly from the command line, 
# jupyter nbconvert --execute "USDA Monthly Report.ipynb" --to asciidoc

from datetime import datetime, timedelta, date
# from urllib.parse import urlencode
import urllib.parse

report_date = '11/30/2023'

end = datetime.strptime(report_date, '%m/%d/%Y')
start = date(end.year, end.month, 1)


# from breeze import breeze
# breeze_api = breeze.BreezeApi(
#     breeze_url = ncc_url,
#     api_key='YourApiKey')

ncc_url = r"https://newmarketchurch.breezechms.com"
base_url = ncc_url + r"/events/reports/243292472#/"


import urllib.parse
params = {
    'start_date': urllib.parse.quote(start.strftime('%m/%d/%Y'), safe='/'),
    'end_date': urllib.parse.quote(end.strftime('%m/%d/%Y'), safe='/'),
    'event_ids': '2949650',
    'attendance_status': 'attended',
    'attendance_operator': 'any',
    'page': 'spreadsheet',
}

print("Download the spreadsheet from the following link. Copy the full path into the 'attendancereport' below.")
print(base_url + urllib.parse.urlencode(params))

Download the spreadsheet from the following link. Copy the full path into the 'attendancereport' below.
https://newmarketchurch.breezechms.com/events/reports/243292472#/start_date=11%2F01%2F2023&end_date=11%2F30%2F2023&event_ids=2949650&attendance_status=attended&attendance_operator=any&page=spreadsheet


In [2]:
# This is the file that was downloaded from https://newmarketchurch.breezechms.com/events/reports/243292472#/&start_date=10%2F01%2F2023&end_date=10%2F31%2F2023&event_ids=2949650&attendance_status=attended&attendance_operator=any&page=spreadsheet

attendancereport = r"C:\Users\ralph\Downloads\FoodPantry-attendance-12-01-2023 (1).xlsx"
peopleexport = r"C:\Users\ralph\Downloads\newmarketchurch-people-12-01-2023 (2).xlsx"

# Paste this file into this month's tab in https://docs.google.com/spreadsheets/d/1hAdgMh7_m73L--bR-rUT2dZPMwEJ8s2e/edit#gid=1826328980
paste_file = r"C:\Users\ralph\Downloads\paste_me.csv"

import numpy as np
import pandas as pd

# Read in the downloaded Excel file 
attendance = pd.read_excel(attendancereport)
people = pd.read_excel(peopleexport)
# print('{count} people in input.'.format(count = len(attendance.index)))

In [3]:
display(attendance.columns)

Index(['Breeze ID', 'First Name', 'Last Name', 'Nov 2, 2023', 'Nov 9, 2023',
       'Nov 16, 2023', 'Nov 23, 2023', 'Nov 30, 2023', 'Person Totals'],
      dtype='object')

In [4]:
attendance = attendance[[
    'Breeze ID', 
    'First Name', 
    'Last Name', 
    'Person Totals'
]]
# drop the Total row
attendance = attendance[attendance['Breeze ID'].notna()]


In [5]:
# Keep only the columns we use.

people = people[[
    'Breeze ID',
    'Date of First Contact',
    'How many people live in your household? (including yourself)',
    'Total number of people aged 18 or under in household?',
    'Total number of people between the ages of 19-59 in household?',
    'Total number of people aged 60 or older in Household?',
    'City'
]]



In [6]:
# Include household numbers from the people profiles.

report = pd.merge(attendance, people, on = 'Breeze ID', how = "inner")
report.fillna(0, inplace=True)

report['Total number of people aged 18 or under in household?'] = report['Total number of people aged 18 or under in household?'].astype(int)
print('{count} people in report.'.format(count = len(report.index)))

# display(report)

187 people in report.


In [7]:
# Clean up the City column.

def clean(word):
    return word.strip().capitalize()
    
report['City'] = report['City'].apply(clean)

In [8]:
# Validate the data 

# Check for numbers in numeric fields
numeric_fields = [
    'Total number of people aged 18 or under in household?',
    'Total number of people between the ages of 19-59 in household?',
    'Total number of people aged 60 or older in Household?',
]
for field in numeric_fields:
    if len(report[~report.map(np.isreal)[field]]) > 0:
        display(field, report[~report.map(np.isreal)[field]])

bad_in_house = report.loc[report['How many people live in your household? (including yourself)'] == 0] 
if len(bad_in_house):
    print("Missing 'Total in house' for")
    display(report.loc[report['How many people live in your household? (including yourself)'] == 0])

bad_totals = report.loc[report['How many people live in your household? (including yourself)'] != 
        report['Total number of people aged 18 or under in household?'] + 
        report['Total number of people between the ages of 19-59 in household?'] + 
        report['Total number of people aged 60 or older in Household?']
    ]
if len(bad_totals) > 0:
    print("Age breakdowns do not add up to total for")
    display(bad_totals)

In [9]:
report.to_csv(paste_file, index=False)

In [10]:
# Create the summary for the USDA monthly report.

MealsPerWeek = 15

numberOfSeniors = 25
SeniorCenter = {
    # 'City': 'Newmarket',
    'Households': numberOfSeniors,
    'individuals': numberOfSeniors,
    'Meals': numberOfSeniors * attendance['Person Totals'].max(),
    'Over 60': numberOfSeniors
}

summary = []

for town in set([t for t in report['City'].unique().flatten()]):
    row = {}
    row['Town'] = town
    filtered = report.loc[report['City'] == town]
    row['Households'] = len(filtered)
    row['individuals'] = filtered['How many people live in your household? (including yourself)'].sum()
    row['Meals'] = MealsPerWeek * (filtered['How many people live in your household? (including yourself)'] * filtered['Person Totals']).sum()
    row['Over 60'] = filtered['Total number of people aged 60 or older in Household?'].sum()
    row['Children'] = filtered['Total number of people aged 18 or under in household?'].sum()
    if town == 'Newmarket':
        for key, value in SeniorCenter.items():
            row[key] += value
    summary.append(row)
    
row = {}
row['Town'] = 'TOTAL'
row['Households'] = len(report)
row['individuals'] = report['How many people live in your household? (including yourself)'].sum()
row['Meals'] = MealsPerWeek * (report['How many people live in your household? (including yourself)'] * report['Person Totals']).sum()
row['Over 60'] = report['Total number of people aged 60 or older in Household?'].sum()
row['Children'] = report['Total number of people aged 18 or under in household?'].sum()
for key, value in SeniorCenter.items():
    row[key] += value

summary.append(row)

pd.DataFrame(summary)

,Town,Households,individuals,Meals,Over 60,Children
0,Salem,1,2,30,1,0
1,Portsmouth,1,1,30,0,0
2,Rye,1,2,120,1,0
3,Dover,1,2,30,0,0
4,Hampton,1,1,15,0,0
5,Newmarket,200,425,14515,108,106
6,Newfields,3,7,270,3,0
7,Epping,1,1,15,0,0
8,Durham,1,1,30,0,0
9,Exeter,1,1,30,0,0
